<a href="https://colab.research.google.com/github/hammaad2002/ASRAdversarialAttacks/blob/huggingface/Tutorial(How_to_use_this_repository).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center> **DISCLAIMER**

This is updated notebook for this branch but since I have done this all work in a rush there might be some bugs/errors. So, please be aware of that!

And I haven't demonstrated here the working hyperparameters of these models also again due to the fact that all this work was done in a rush so sorry for the inconvenience :(

In [3]:
%%capture
!git clone -b huggingface https://github.com/hammaad2002/ASRAdversarialAttacks.git
!git clone https://github.com/hammaad2002/CRDNN_Model.git
!pip install transformers

In [5]:
from ASRAdversarialAttacks.AdversarialAttacks import ASRAttacks
import torchaudio
import torch
import numpy as np
from IPython.display import Audio

In [6]:
# Checking the device available during the current environment (CUDA is recommended!)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = "facebook/wav2vec2-base-960h"

In [7]:
# Loading the audio
input_audio, sample_rate = torchaudio.load('/content/CRDNN_Model/AudioSamplesASR/spk1_snt1.wav')

In [8]:
# My target
target_transcription = 'WHEN MAN NOTHING KILL IN BIG CAMEL'
true_transcription = 'THE CHILD ALMOST HURT THE SMALL DOG'

In [9]:
#initializing attack class
attack = ASRAttacks(model, device)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# <center>**FGSM ATTACK**</center>

```
        Simple fast gradient sign method attack is implemented which is the simplest
        adversarial attack in this domain.
        For more information, see the paper: https://arxiv.org/pdf/1412.6572.pdf

        INPUT ARGUMENTS:

        input__       : Input audio. Ex: Tensor[0.1,0.3,...] or (samples,)
                        Type: torch.Tensor

        target        : Target transcription (needed if the you want targeted
                        attack) Ex: ["my name is mango."].
                        Type: List[str]
                        CAUTION:
                        Please make sure these characters are also present in the
                        dictionary of the model also.

        epsilon       : Noise controlling parameter.
                        Type: float

        targeted      : If the attack should be targeted towards your desired
                        transcription or not.
                        Type: bool
                        CAUTION:
                        Please make to pass your targetted
                        transcription also in this case).

        RETURNS:

        np.ndarray : Perturbed audio
```


# **FGSM TARGETED**

In [10]:
#FGSM
temp1 = attack.FGSM_ATTACK(input_audio, target_transcription, epsilon = 0.01, targeted = True)

#FGSM PRINT
print(attack.INFER(torch.from_numpy(temp1)))
print(target_transcription)

A CHILD ALMOST HURT A SMALL DOG
WHEN MAN NOTHING KILL IN BIG CAMEL


For WER use [0] and for in depth information of insertion, selection, and deletion use [1]

In [11]:
print("Targeted WER is: ", attack.wer_compute([target_transcription], [temp1], targeted= True)[0])

Targeted WER is:  0.0


In [12]:
info = attack.wer_compute([target_transcription], [temp1], targeted= True)[1]
print(f"Insertion: {info[0][1]}, Substitution: {info[0][0]}, Deletion: {info[0][2]}")

Insertion: 0, Substitution: 7, Deletion: 0


In [13]:
Audio(temp1, rate =16000)

# **FGSM UNTARGETED**

In [14]:
#FGSM
temp2 = attack.FGSM_ATTACK(input_audio, epsilon = 0.0125, targeted = False)

#FGSM PRINT
print(attack.INFER(torch.from_numpy(temp2)))

THE CHILD ALMOST HER I NOW DOG


In [15]:
print("Untargeted WER is: ", attack.wer_compute([target_transcription], [temp2], targeted= False)[0])

Untargeted WER is:  1.0


In [16]:
info = attack.wer_compute([target_transcription], [temp2], targeted= False)[1]
print(f"Insertion: {info[0][1]}, Substitution: {info[0][0]}, Deletion: {info[0][2]}")

Insertion: 0, Substitution: 7, Deletion: 0


In [17]:
Audio(temp2, rate =16000)

# <center>**BIM ATTACK**</center>

```
        Basic Itertive Moment attack is implemented which is simple Fast Gradient
        Sign Attack but in loop.
        For more information, see the paper: https://arxiv.org/abs/1607.02533

        INPUT ARGUMENTS:

        input__       : Input audio. Ex: Tensor[0.1,0.3,...] or (samples,)
                        Type: torch.Tensor

        target        : Target transcription (needed if the you want targeted
                        attack) Ex: ["my name is mango."].
                        Type: List[str]
                        CAUTION:
                        Please make sure these characters are also present in the
                        dictionary of the model also.

        epsilon       : Noise controlling parameter.
                        Type: float

        alpha         : Noise contribution in input controlling parameter
                        Type: float

        num_iter      : Number of iteration of attack
                        Type: int

        targeted      : If the attack should be targeted towards your desired
                        transcription or not.
                        Type: bool
                        CAUTION:
                        Please make to pass your targetted
                        transcription also in this case).

        early_stop    : If user wants to stop the attack early if the attack reaches the target transcription before the total number of iterations.
                        Type: bool
                        
        RETURNS:
        
        np.ndarray : Perturbed audio
```


# **BIM TARGETED**

With early stopping:

In [18]:
#BIM
temp3 = attack.BIM_ATTACK(input_audio, target_transcription, epsilon = 0.0015, alpha = 0.00009,
                   num_iter = 3, targeted = True, early_stop = True)

#BIM PRINT
print(attack.INFER(torch.from_numpy(temp3)))
print(target_transcription)

  0%|          | 0/3 [00:00<?, ?it/s]

THE CHILD ALMOST HURT THE SMALL DOG
WHEN MAN NOTHING KILL IN BIG CAMEL


In [19]:
print("Targeted WER is: ", attack.wer_compute([target_transcription], [temp3], targeted= True)[0])

Targeted WER is:  0.0


In [20]:
info = attack.wer_compute([target_transcription], [temp3], targeted= True)[1]
print(f"Insertion: {info[0][1]}, Substitution: {info[0][0]}, Deletion: {info[0][2]}")

Insertion: 0, Substitution: 7, Deletion: 0


In [21]:
Audio(temp3, rate = 16000)

Without early stopping:

In [22]:
#BIM
temp4 = attack.BIM_ATTACK(input_audio, target_transcription, epsilon = 0.0015, alpha = 0.00009,
                   num_iter = 3, targeted = True, early_stop = False)

#BIM PRINT
print(attack.INFER(torch.from_numpy(temp4)))
print(target_transcription)

  0%|          | 0/3 [00:00<?, ?it/s]

THE CHILD ALMOST HURT THE SMALL DOG
WHEN MAN NOTHING KILL IN BIG CAMEL


In [23]:
print("Targeted WER is: ", attack.wer_compute([target_transcription], [temp4], targeted= True)[0])

Targeted WER is:  0.0


In [24]:
info = attack.wer_compute([target_transcription], [temp4], targeted= True)[1]
print(f"Insertion: {info[0][1]}, Substitution: {info[0][0]}, Deletion: {info[0][2]}")

Insertion: 0, Substitution: 7, Deletion: 0


In [25]:
Audio(temp4, rate = 16000)

# **BIM UNTARGETED**

With early stopping:

In [26]:
#BIM
temp5 = attack.BIM_ATTACK(input_audio, epsilon = 0.0015, alpha = 0.00009,
                   num_iter = 2, targeted = False, early_stop = True)

#BIM PRINT
print(attack.INFER(torch.from_numpy(temp5)))

  0%|          | 0/2 [00:00<?, ?it/s]

THE CHILD ALMOST HURT THE SMALL DOG


In [27]:
print("Untargeted WER is: ", attack.wer_compute([target_transcription], [temp5], targeted= False)[0])

Untargeted WER is:  1.0


In [28]:
info = attack.wer_compute([target_transcription], [temp5], targeted= False)[1]
print(f"Insertion: {info[0][1]}, Substitution: {info[0][0]}, Deletion: {info[0][2]}")

Insertion: 0, Substitution: 7, Deletion: 0


In [29]:
Audio(temp5, rate =16000)

Without early stopping:

In [30]:
#BIM
temp6 = attack.BIM_ATTACK(input_audio, epsilon = 0.0015, alpha = 0.00009,
                   num_iter = 2, targeted = False, early_stop = False)

#BIM PRINT
print(attack.INFER(torch.from_numpy(temp6)))

  0%|          | 0/2 [00:00<?, ?it/s]

THE CHILD ALMOST HURT THE SMALL DOG


In [31]:
print("Untargeted WER is: ", attack.wer_compute([target_transcription], [temp6], targeted= False)[0])

Untargeted WER is:  1.0


In [32]:
info = attack.wer_compute([target_transcription], [temp6], targeted= False)[1]
print(f"Insertion: {info[0][1]}, Substitution: {info[0][0]}, Deletion: {info[0][2]}")

Insertion: 0, Substitution: 7, Deletion: 0


In [33]:
Audio(temp6, rate =16000)

# <center>**PGD ATTACK**</center>

```
        Projected Gradient Descent attack is implemented which in simple terms is more
        advanced version of BIM. In this attack we project our perturbation back to
        some Lp norm and find perturbations in that particular region.
        For more information, see the paper: https://arxiv.org/abs/1706.06083

        INPUT ARGUMENTS:

        input__       : Input audio. Ex: Tensor[0.1,0.3,...] or (samples,)
                        Type: torch.Tensor

        target        : Target transcription (needed if the you want targeted
                        attack) Ex: ["my name is mango."].
                        Type: List[str]
                        CAUTION:
                        Please make sure these characters are also present in the
                        dictionary of the model also.

        epsilon       : Noise controlling parameter.
                        Type: float

        alpha         : Noise contribution in input controlling parameter
                        Type: float

        num_iter      : Number of iteration of attack
                        Type: int

        targeted      : If the attack should be targeted towards your desired
                        transcription or not.
                        Type: bool
                        CAUTION:
                        Please make to pass your targetted
                        transcription also in this case).

        early_stop    : If user wants to stop the attack early if the attack reaches the target transcription before the total number of iterations.
                        Type: bool
                        
        RETURNS:
        
        np.ndarray : Perturbed audio
```

# **PGD TARGETED**

With early stopping:

In [34]:
#PGD
temp7 = attack.PGD_ATTACK(input_audio, target_transcription, epsilon = 0.0015, alpha = 0.00009,
                   num_iter = 2, targeted = True, early_stop = True)

#PGD PRINT
print(attack.INFER(torch.from_numpy(temp7)))
print(target_transcription)

  0%|          | 0/2 [00:00<?, ?it/s]

THE CHILD ALMOST HURT THE SMALL DOG
WHEN MAN NOTHING KILL IN BIG CAMEL


In [35]:
print("Targeted WER is: ", attack.wer_compute([target_transcription], [temp7], targeted= True)[0])

Targeted WER is:  0.0


In [36]:
info = attack.wer_compute([target_transcription], [temp7], targeted= True)[1]
print(f"Insertion: {info[0][1]}, Substitution: {info[0][0]}, Deletion: {info[0][2]}")

Insertion: 0, Substitution: 7, Deletion: 0


In [37]:
Audio(temp7, rate =16000)

Without early stopping:

In [38]:
#PGD
temp8 = attack.PGD_ATTACK(input_audio, target_transcription, epsilon = 0.0015, alpha = 0.00009,
                   num_iter = 2, targeted = True, early_stop = False)

#PGD PRINT
print(attack.INFER(torch.from_numpy(temp8)))
print(target_transcription)

  0%|          | 0/2 [00:00<?, ?it/s]

THE CHILD ALMOST HURT THE SMALL DOG
WHEN MAN NOTHING KILL IN BIG CAMEL


In [39]:
print("Targeted WER is: ", attack.wer_compute([target_transcription], [temp8], targeted= True)[0])

Targeted WER is:  0.0


In [40]:
info = attack.wer_compute([target_transcription], [temp8], targeted= True)[1]
print(f"Insertion: {info[0][1]}, Substitution: {info[0][0]}, Deletion: {info[0][2]}")

Insertion: 0, Substitution: 7, Deletion: 0


In [41]:
Audio(temp8, rate =16000)

# **PGD UNTARGETED**

With early stopping:

In [42]:
#PGD
temp9 = attack.PGD_ATTACK(input_audio, epsilon = 0.0015, alpha = 0.0001,
                   num_iter = 1, targeted = False, early_stop = True)

#PGD PRINT
print(attack.INFER(torch.from_numpy(temp9)))

  0%|          | 0/1 [00:00<?, ?it/s]

HE CHILD ALMOST HURT THE SMALL DOG


In [43]:
print("Untargeted WER is: ", attack.wer_compute([target_transcription], [temp9], targeted= False)[0])

Untargeted WER is:  1.0


In [44]:
info = attack.wer_compute([target_transcription], [temp9], targeted= False)[1]
print(f"Insertion: {info[0][1]}, Substitution: {info[0][0]}, Deletion: {info[0][2]}")

Insertion: 0, Substitution: 7, Deletion: 0


In [45]:
Audio(temp9, rate =16000)

Without early stopping:

In [46]:
#PGD
temp10 = attack.PGD_ATTACK(input_audio, epsilon = 0.0015, alpha = 0.0001,
                   num_iter = 1, targeted = False, early_stop = True)

#PGD PRINT
print(attack.INFER(torch.from_numpy(temp10)))

  0%|          | 0/1 [00:00<?, ?it/s]

HE CHILD ALMOST HURT THE SMALL DOG


In [47]:
print("Untargeted WER is: ", attack.wer_compute([target_transcription], [temp10], targeted= False)[0])

Untargeted WER is:  1.0


In [48]:
info = attack.wer_compute([target_transcription], [temp10], targeted= False)[1]
print(f"Insertion: {info[0][1]}, Substitution: {info[0][0]}, Deletion: {info[0][2]}")

Insertion: 0, Substitution: 7, Deletion: 0


In [49]:
Audio(temp10, rate =16000)

# <center>**CW ATTACK**</center>

```
        Implements the Carlini and Wagner attack, the strongest white box
        adversarial attack. This attack uses an optimization strategy to find the
        adversarial transcription while keeping the perturbation as low as possible.
        For more information, see the paper: https://arxiv.org/pdf/1801.01944.pdf
        
        INPUT ARGUMENTS:
        
        input__       : Input audio. Ex: Tensor[0.1,0.3,...] or (samples,)
                        Type: torch.Tensor
                        
        target        : Target transcription (needed if the you want targeted
                        attack) Ex: ["my name is mango."].
                        Type: List[str]
                        CAUTION:
                        Please make sure these characters are also present in the
                        dictionary of the model also.
                        
        epsilon       : Noise controlling parameter.
                        Type: float
                        
        c             : Regularization term controlling factor.
                        Type: float
                        
        learning_rate : learning_rate of optimizer.
                        Type: float
                        
        num_iter      : Number of iteration of attack.
                        Type: int
                        
        decrease_factor_eps   : Factor to decrease epsilon during search
                                Type: float
                                
        num_iter_decrease_eps : Number of iterations after which to decrease epsilon
                                Type: int
                                
        optimizer     : Name of the optimizer to use for the attack.
                        Type: str
                        
        nested        : if this attack in being run in a for loop with tqdm
                        Type: bool
                        
        early_stop    : if the user wants to end the attack as soon as the attack
                        gets the target transcription.
                        Type: bool
                        
        search_eps    : if the user wants the attack to search for the epsilon value
                        on its own provided the initial epsilon value of large.
                        Type: bool
        
        targeted      : If the attack should be targeted towards your desired
                        transcription or not.
                        Type: bool
                        CAUTION:
                        Please make to pass your targetted
                        transcription also in this case).
                        
        RETURNS:
        
        np.ndarray : Perturbed audio
```

# **CW TARGETED**

With early stopping:

In [50]:
#CW
temp11 = attack.CW_ATTACK(input_audio, target_transcription, epsilon = 0.0015, c = 10,
                  learning_rate = 0.00001, num_iter = 1, decrease_factor_eps = 1,
                  num_iter_decrease_eps = 10, optimizer = None, nested = True,
                  early_stop = True, search_eps = False, targeted = True)

#CW PRINT
print(attack.INFER(torch.from_numpy(temp11)))
print(target_transcription)

  0%|          | 0/1 [00:00<?, ?it/s]

THE CHILD ALMOST HURT THE SMALL DOG
WHEN MAN NOTHING KILL IN BIG CAMEL


In [51]:
print("Targeted WER is: ", attack.wer_compute([target_transcription], [temp11], targeted= True)[0])

Targeted WER is:  0.0


In [52]:
info = attack.wer_compute([target_transcription], [temp11], targeted= True)[1]
print(f"Insertion: {info[0][1]}, Substitution: {info[0][0]}, Deletion: {info[0][2]}")

Insertion: 0, Substitution: 7, Deletion: 0


In [53]:
Audio(temp11, rate =16000)

Without early stopping:

In [54]:
#CW
temp12 = attack.CW_ATTACK(input_audio, target_transcription, epsilon = 0.0015, c = 10,
                  learning_rate = 0.00001, num_iter = 1, decrease_factor_eps = 1,
                  num_iter_decrease_eps = 10, optimizer = None, nested = True,
                  early_stop = False, search_eps = False, targeted = True)

#CW PRINT
print(attack.INFER(torch.from_numpy(temp12)))
print(target_transcription)

  0%|          | 0/1 [00:00<?, ?it/s]

THE CHILD ALMOST HURT THE SMALL DOG
WHEN MAN NOTHING KILL IN BIG CAMEL


In [55]:
print("Targeted WER is: ", attack.wer_compute([target_transcription], [temp12], targeted= True)[0])

Targeted WER is:  0.0


In [56]:
info = attack.wer_compute([target_transcription], [temp12], targeted= True)[1]
print(f"Insertion: {info[0][1]}, Substitution: {info[0][0]}, Deletion: {info[0][2]}")

Insertion: 0, Substitution: 7, Deletion: 0


In [57]:
Audio(temp12, rate =16000)

# **CW UNTARGETED**

With early stopping:

In [58]:
#CW
temp13 = attack.CW_ATTACK(input_audio, epsilon = 0.0015, c = 10,
                  learning_rate = 0.00001, num_iter = 1, decrease_factor_eps = 1,
                  num_iter_decrease_eps = 10, optimizer = None, nested = True,
                  early_stop = True, search_eps = False, targeted = False)

#CW PRINT
print(attack.INFER(torch.from_numpy(temp13)))

  0%|          | 0/1 [00:00<?, ?it/s]

THE CHILD ALMOST HURT TTHE SMALL DOG


In [59]:
print("Untargeted WER is: ", attack.wer_compute([target_transcription], [temp13], targeted= False)[0])

Untargeted WER is:  1.0


In [60]:
info = attack.wer_compute([target_transcription], [temp13], targeted= False)[1]
print(f"Insertion: {info[0][1]}, Substitution: {info[0][0]}, Deletion: {info[0][2]}")

Insertion: 0, Substitution: 7, Deletion: 0


In [61]:
Audio(temp13, rate =16000)

Without early stopping:

In [62]:
#CW
temp14 = attack.CW_ATTACK(input_audio, epsilon = 0.0015, c = 10,
                  learning_rate = 0.00001, num_iter = 1, decrease_factor_eps = 1,
                  num_iter_decrease_eps = 10, optimizer = None, nested = True,
                  early_stop = False, search_eps = False, targeted = False)

#CW PRINT
print(attack.INFER(torch.from_numpy(temp14)))

  0%|          | 0/1 [00:00<?, ?it/s]

THE CHILD ALMOST HURT TTHE SMALL DOG


In [63]:
print("Untargeted WER is: ", attack.wer_compute([target_transcription], [temp14], targeted= False)[0])

Untargeted WER is:  1.0


In [64]:
info = attack.wer_compute([target_transcription], [temp14], targeted= False)[1]
print(f"Insertion: {info[0][1]}, Substitution: {info[0][0]}, Deletion: {info[0][2]}")

Insertion: 0, Substitution: 7, Deletion: 0


In [65]:
Audio(temp14, rate = 16000)

# <center>**IMPERCEPTIBLE ASR ATTACK**</center>


```
        Implements the Imperceptible ASR attack, which leverages the strongest white box
        adversarial attack which is CW attack while also masking sure the added perturbation
        is imperceptible to humans using Psychoacoustic Scale. This attack is performed in two
        stages. In first stage we perform simple CW attack and in 2nd stage we make sure our
        added perturbations are imperceptible.
        For more information, see the paper: https://arxiv.org/abs/1903.10346
        
        INPUT ARGUMENTS:
        
        input__       : Input audio. Ex: Tensor[0.1,0.3,...] or (samples,)
                        Type: torch.Tensor
                        
        target        : Target transcription (needed if the you want targeted
                        attack) Ex: ["my name is mango."]
                        Type: List[str]
                        CAUTION:
                        Please make sure these characters are also present in the
                        dictionary of the model also
                        
        epsilon       : Noise controlling parameter
                        Type: float
                        
        c             : Regularization term controlling factor
                        Type: float
                        
        learning_rate1: learning_rate of optimizer for stage 1
                        Type: float
        
        learning_rate2: learning_rate of optimizer for stage 2
                        Type: float
                        
        num_iter1     : Number of iteration of attack stage 1
                        Type: int
        
        num_iter2     : Number of iteration of attack stage 2
                        Type: int
                        
        decrease_factor_eps   : Factor to decrease epsilon by during optimization
                                Type: float
                                
        num_iter_decrease_eps : Number of iterations after which to decrease epsilon
                                Type: int
                                
        optimizer1     : Name of the optimizer to use for the attack stage 1
                         Type: str
                         
        optimizer2     : Name of the optimizer to use for the attack stage 2
                         Type: str
                         
        nested         : if this attack in being run in a for loop with tqdm
                         Type: bool
        
        early_stop_cw  : if the user wants 1st stage attack to early stop or not
                         Type: bool
        
        search_eps_cw  : if the user wants 1st stage attack to search for epsilon
                         value provided a large intial epsilon value is provided
                         Type: bool
                         
        alpha          : regularization term for second stage loss
                         Type: float
                         
        RETURNS:
        
        np.ndarray     : Perturbed audio
```

With early stopping:

In [66]:
temp15 = attack.IMPERCEPTIBLE_ATTACK(torch.nn.functional.pad(input_audio, (0, 1000)), target_transcription, epsilon = 0.015, c = 10, learning_rate1 = 0.001,
                                    learning_rate2 = 0.0001, num_iter1 = 1, num_iter2 = 1, decrease_factor_eps = 1,
                                    num_iter_decrease_eps = 10, optimizer1 = None, optimizer2 = "Adam",nested = True ,
                                    early_stop_cw = True, search_eps_cw = False, alpha = 0.05)

# Attack's transcription
print(attack.INFER(torch.from_numpy(temp15)))

*****Attack Stage 1*****:   0%|          | 0/1 [00:00<?, ?it/s]

*****Attack Stage 2*****:   0%|          | 0/1 [00:00<?, ?it/s]

THE CHILD ALMOST HER A CNABOB


In [67]:
print("Targeted WER is: ", attack.wer_compute([target_transcription], [temp15], targeted= True)[0])

Targeted WER is:  0.0


In [68]:
info = attack.wer_compute([target_transcription], [temp15], targeted= True)[1]
print(f"Insertion: {info[0][1]}, Substitution: {info[0][0]}, Deletion: {info[0][2]}")

Insertion: 0, Substitution: 6, Deletion: 1


In [69]:
Audio(temp15[:,:-1000], rate = 16000)

Without early stopping:

In [70]:
temp16 = attack.IMPERCEPTIBLE_ATTACK(torch.nn.functional.pad(input_audio, (0, 1000)), target_transcription, epsilon = 0.015, c = 10, learning_rate1 = 0.001,
                                    learning_rate2 = 0.0001, num_iter1 = 1, num_iter2 = 1, decrease_factor_eps = 1,
                                    num_iter_decrease_eps = 10, optimizer1 = None, optimizer2 = "Adam",nested = True ,
                                    early_stop_cw = False, search_eps_cw = False, alpha = 0.05)

# Attack's transcription
print(attack.INFER(torch.from_numpy(temp16)))

*****Attack Stage 1*****:   0%|          | 0/1 [00:00<?, ?it/s]

*****Attack Stage 2*****:   0%|          | 0/1 [00:00<?, ?it/s]

THE CHILD ALMOST HER A CNABOB


In [71]:
print("Targeted WER is: ", attack.wer_compute([target_transcription], [temp16], targeted= True)[0])

Targeted WER is:  0.0


In [72]:
info = attack.wer_compute([target_transcription], [temp16], targeted= True)[1]
print(f"Insertion: {info[0][1]}, Substitution: {info[0][0]}, Deletion: {info[0][2]}")

Insertion: 0, Substitution: 6, Deletion: 1


In [73]:
Audio(temp16[:,:-1000], rate = 16000)

----------------------------

# <center> **COMPARISON**

METRIC I USED FOR COMPARISON

In [74]:
# SNR metric for comparison to check which attack adds less noise
def Metricsnr(original, noisy):
    original_power = 20 * np.log10(np.mean(original ** 2))
    noise_power = 20 * np.log10(np.mean((original - noisy) ** 2))
    snr = noise_power - original_power
    return snr

In [75]:
orig = input_audio.numpy() # clean audio
orig.shape

(1, 45920)

-------------------

# **TARGETED ATTACKS**

With early stopping

In [76]:
Audio(temp3, rate = 16000) #BIM targeted

In [77]:
Audio(temp7, rate = 16000) #PGD targeted

In [78]:
Audio(temp11, rate = 16000) #CW targeted

In [79]:
Audio(temp15[:,:-1000], rate = 16000) # Imperceptible ASR

In [80]:
print("LOWER SNR IS BETTER OVER BECAUSE OF THE WAY IT IS MEASURED")
print("BIM SNR                  :",Metricsnr(orig, temp3))
print("PGD SNR                  :",Metricsnr(orig, temp7))
print("CW SNR                   :",Metricsnr(orig, temp11))
print("IMPERCEPTIBLE ATTACK SNR :",Metricsnr(orig, temp15[:,:-1000]))

LOWER SNR IS BETTER OVER BECAUSE OF THE WAY IT IS MEASURED
BIM SNR                  : -85.97746849060059
PGD SNR                  : -89.76689338684082
CW SNR                   : -58.18476676940918
IMPERCEPTIBLE ATTACK SNR : -7.580461502075195


Without early stopping

In [81]:
Audio(temp4, rate = 16000) #BIM targeted

In [82]:
Audio(temp8, rate = 16000) #PGD targeted

In [83]:
Audio(temp12, rate = 16000) #CW targeted

In [84]:
Audio(temp16[:,:-1000], rate = 16000) # Imperceptible ASR

In [85]:
print("LOWER SNR IS BETTER OVER BECAUSE OF THE WAY IT IS MEASURED")
print("BIM SNR                  :",Metricsnr(orig, temp4))
print("PGD SNR                  :",Metricsnr(orig, temp8))
print("CW SNR                   :",Metricsnr(orig, temp12))
print("IMPERCEPTIBLE ATTACK SNR :",Metricsnr(orig, temp16[:,:-1000]))

LOWER SNR IS BETTER OVER BECAUSE OF THE WAY IT IS MEASURED
BIM SNR                  : -85.97746849060059
PGD SNR                  : -89.76689338684082
CW SNR                   : -58.18476676940918
IMPERCEPTIBLE ATTACK SNR : -7.580461502075195


-------------------

-------------------

# **UNTARGETED ATTACKS**

With early stopping

In [86]:
Audio(temp5, rate = 16000) #BIM untargeted

In [87]:
Audio(temp9, rate = 16000) #PGD untargeted

In [88]:
Audio(temp13, rate = 16000) #CW untargeted

In [89]:
print("LOWER SNR IS BETTER OVER BECAUSE OF THE WAY IT IS MEASURED")
print("BIM SNR:",Metricsnr(orig, temp5))
print("PGD SNR:",Metricsnr(orig, temp9))
print("CW SNR :",Metricsnr(orig, temp13))

LOWER SNR IS BETTER OVER BECAUSE OF THE WAY IT IS MEASURED
BIM SNR: -90.12113571166992
PGD SNR: -94.40433502197266
CW SNR : -57.9774284362793


Without early stopping

In [90]:
Audio(temp6, rate = 16000) #BIM untargeted

In [91]:
Audio(temp10, rate = 16000) #PGD untargeted

In [92]:
Audio(temp14, rate = 16000) #PGD untargeted

In [93]:
print("LOWER SNR IS BETTER OVER BECAUSE OF THE WAY IT IS MEASURED")
print("BIM SNR:",Metricsnr(orig, temp6))
print("PGD SNR:",Metricsnr(orig, temp10))
print("CW SNR :",Metricsnr(orig, temp14))

LOWER SNR IS BETTER OVER BECAUSE OF THE WAY IT IS MEASURED
BIM SNR: -90.12113571166992
PGD SNR: -94.40433502197266
CW SNR : -57.9774284362793


-------------------